# Introduction

This scripts show how to generate with huggingface generate and vllm

In [32]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'
os.environ['HF_HUB_CACHE'] = '/next_share/hf_cache/hub'

import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, GenerationMixin, T5ForConditionalGeneration
from vllm import LLM, SamplingParams

## Generate with VLLM

In [3]:
engine = LLM('meta-llama/Meta-Llama-3-8B')

INFO 05-17 16:22:55 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='meta-llama/Meta-Llama-3-8B', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B)


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

INFO 05-17 16:22:58 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 05-17 16:22:58 selector.py:32] Using XFormers backend.
INFO 05-17 16:22:59 weight_utils.py:199] Using model weights format ['*.safetensors']
INFO 05-17 16:23:42 model_runner.py:175] Loading model weights took 14.9595 GB
INFO 05-17 16:23:44 gpu_executor.py:114] # GPU blocks: 2423, # CPU blocks: 2048
INFO 05-17 16:23:47 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-17 16:23:47 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 05-17 16:23:52

In [108]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.0, top_p=0.95, max_tokens=300, ignore_eos=True)

In [109]:
# outputs = [engine.generate(prompt, sampling_params, use_tqdm=False)[0] for prompt in tqdm(prompts)]
outputs = engine.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 4/4 [00:07<00:00,  1.86s/it]


In [54]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Prompt: 'Hello, my name is', Generated text: " and I'm writing you today to learn more about the 2019 Chevrolet Silverado 1500 LTZ Crew Cab 4WD. I live at in the area and I would like to hear back from you soon and learn more about this vehicle. Please call me at at your earliest convenience.://www.chevrolet.com/://www.chevrolet.com/://www.chevrolet.com/://www.chevrolet.com/ - 2019 Chevrolet Silverado 1500 LTZ Crew Cab 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4WD - 4

In [91]:
print(outputs[2].outputs[0].text)

 Paris. It is located in the north of the country. The city is situated on the banks of the Seine River. Paris is the largest city in France. It is also the largest city in the European Union. The city has a population of over 2.2 million people. Paris is a major tourist destination. It is home to many famous landmarks, including the Eiffel Tower, the Louvre Museum, and the Notre Dame Cathedral. The city is also a major center of culture and the arts. Paris is home to many famous artists, writers, and musicians. The city is also a major center of business and finance. Paris is a major transportation hub. It is served by two international airports, Charles de Gaulle and Orly. The city is also served by a major railway station, Gare de Lyon. Paris is a major center of education. It is home to many universities and colleges. The city is also home to many research institutes and laboratories. Paris is a major center of science and technology. The city is home to many major companies, inclu

In [50]:
sampling_params

SamplingParams(n=1, best_of=1, presence_penalty=0.0, frequency_penalty=0.0, repetition_penalty=1.0, temperature=0.0, top_p=1.0, top_k=-1, min_p=0.0, seed=None, use_beam_search=False, length_penalty=1.0, early_stopping=False, stop=[], stop_token_ids=[], include_stop_str_in_output=False, ignore_eos=False, max_tokens=800, min_tokens=0, logprobs=None, prompt_logprobs=None, skip_special_tokens=True, spaces_between_special_tokens=True, truncate_prompt_tokens=None)

In [92]:
print([len(output.outputs[0].token_ids) for output in outputs])

[512, 512, 512, 512]


## Generate with Transformers

In [33]:
model = AutoModelForCausalLM.from_pretrained('meta-llama/Meta-Llama-3-8B', torch_dtype = torch.bfloat16, device_map = 1)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [75]:
tk = engine.get_tokenizer()
print(tk.eos_token)
tk.pad_token = tk.eos_token
tk.padding_side = 'left'

<|end_of_text|>


In [44]:
inputs = [tk(p, return_tensors='pt') for p in prompts]

In [72]:
def to_cuda(kws, gpu=0):
    return {k:v.cuda(gpu) for k,v in kws.items()}

In [67]:
all_outs = []
for ipt in tqdm(inputs):
    r = model.generate(**to_cuda(ipt, 1), 
                       do_sample = False, max_new_tokens = 512, 
                       eos_token_id = -1,
                       pad_token_id = tk.eos_token_id)
    all_outs.append(r[0])

  0%|          | 0/4 [00:00<?, ?it/s]/storage_fast/rhshui/lib/anaconda3/envs/llm2/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/storage_fast/rhshui/lib/anaconda3/envs/llm2/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 4/4 [01:01<00:00, 15.44s/it]


In [110]:
enc = tk(prompts, return_tensors = 'pt', padding = True)
r_ba = model.generate(**to_cuda(enc, 1),
                      do_sample = False, max_new_tokens = 300, 
                       eos_token_id = -1,
                       pad_token_id = tk.eos_token_id)

In [93]:
print([len(k) for k in r_ba])

[520, 520, 520, 520]


In [87]:
print(tk.decode(r_ba[2]))

<|end_of_text|><|end_of_text|><|begin_of_text|>The capital of France is Paris. It is located in the north of the country. The city is situated on the banks of the Seine River. Paris is the largest city in France. It is also the largest city in the European Union. The city has a population of over 2.2 million people. Paris is a major tourist destination. It is home to many famous landmarks, including the Eiffel Tower, the Louvre Museum, and the Notre Dame Cathedral. The city is also a major center of culture and the arts. Paris is home to many world-renowned museums, galleries, and theaters. The city is also a major center of fashion and design. Paris is a major center of business and finance. The city is home to many multinational corporations and financial institutions. Paris is a major transportation hub. The city is served by two international airports and a major railway station. Paris is a major center of education. The city is home to many universities and colleges. Paris is a ma

In [49]:
tk.eos_token_id

128001

In [97]:
enc.attention_mask[2]

tensor([0, 0, 1, 1, 1, 1, 1, 1])

In [99]:
a = r_ba[2][8:]
b = outputs[2].outputs[0].token_ids
print(len(a), len(b))

512 512


In [100]:
print(tk.decode(a))

 Paris. It is located in the north of the country. The city is situated on the banks of the Seine River. Paris is the largest city in France. It is also the largest city in the European Union. The city has a population of over 2.2 million people. Paris is a major tourist destination. It is home to many famous landmarks, including the Eiffel Tower, the Louvre Museum, and the Notre Dame Cathedral. The city is also a major center of culture and the arts. Paris is home to many world-renowned museums, galleries, and theaters. The city is also a major center of fashion and design. Paris is a major center of business and finance. The city is home to many multinational corporations and financial institutions. Paris is a major transportation hub. The city is served by two international airports and a major railway station. Paris is a major center of education. The city is home to many universities and colleges. Paris is a major center of research and development. The city is home to many resear

In [101]:
print(tk.decode(b))

 Paris. It is located in the north of the country. The city is situated on the banks of the Seine River. Paris is the largest city in France. It is also the largest city in the European Union. The city has a population of over 2.2 million people. Paris is a major tourist destination. It is home to many famous landmarks, including the Eiffel Tower, the Louvre Museum, and the Notre Dame Cathedral. The city is also a major center of culture and the arts. Paris is home to many famous artists, writers, and musicians. The city is also a major center of business and finance. Paris is a major transportation hub. It is served by two international airports, Charles de Gaulle and Orly. The city is also served by a major railway station, Gare de Lyon. Paris is a major center of education. It is home to many universities and colleges. The city is also home to many research institutes and laboratories. Paris is a major center of science and technology. The city is home to many major companies, inclu